In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain.prompts.chat import ChatPromptTemplate

In [3]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    model="llama3.1",
    api_key="ollama",
    base_url="http://127.0.0.1:11434/v1"
)

![IMG](./image/5.png)

In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [6]:
from pydantic import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [7]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])

In [8]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [9]:
def research_plan_node(state: AgentState):

    prompt = ChatPromptTemplate.from_messages([
        ("system", RESEARCH_PLAN_PROMPT),
        ("placeholder", "{messages}")
    ])
    structured_model = model.with_structured_output(Queries, strict=True)
    chain = prompt | structured_model
    queries = chain.invoke({"messages": [state['task']]})

    if "content" in state.keys():
        content = state['content']
    else:
        content = []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [10]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }

In [11]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [12]:
def research_critique_node(state: AgentState):

    prompt = ChatPromptTemplate.from_messages([
        ("system", RESEARCH_CRITIQUE_PROMPT),
        ("placeholder", "{messages}")
    ])
    structured_model = model.with_structured_output(Queries, strict=True)
    chain = prompt | structured_model
    queries = chain.invoke({"messages": [state['critique']]})

    if "content" in state.keys():
        content = state['content']
    else:
        content = []
    
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [13]:
def should_continue(state):
    if state['revision_number'] > state['max_revisions']:
        return END
    return "reflect"

In [14]:
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")

builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)

builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [15]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    thread = {"configurable": {"thread_id": "1"}}
    graph = builder.compile(checkpointer=memory)
    for s in graph.stream({
        'task': "what is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    }, thread):
        print(s)

{'planner': {'plan': 'LangChain and LangSmith are both AI platforms that enable developers to build, deploy, and manage conversational AI models, but they have distinct differences in their focus, features, and use cases.\n\nHere\'s a high-level outline of an essay comparing LangChain and LangSmith:\n\n**Title:** "A Comparative Analysis of LangChain and LangSmith: Building the Future of Conversational AI"\n\n**I. Introduction**\n\n* Brief overview of LangChain and LangSmith\n* Importance of conversational AI in modern applications\n* Thesis statement: While both LangChain and LangSmith are powerful tools for building conversational AI, they differ significantly in their approach, capabilities, and use cases.\n\n**II. Overview of LangChain**\n\n* Description of LangChain as a platform for building, deploying, and managing conversational AI models\n* Focus on ease of use, flexibility, and scalability\n* Examples of applications where LangChain is suitable (e.g., chatbots, voice assistant